**** CREDITS: ****

https://www.youtube.com/watch?v=eaa-7HLcAnQ
    
https://www.kaggle.com/gcdatkin/concrete-crack-image-detection

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
positive_dir = Path('/home/pratham/study/input_data/kaggle_surface_crack_detection/Positive/')
negative_dir = Path('/home/pratham/study/input_data/kaggle_surface_crack_detection/Negative/')

# Creating DataFrames

In [3]:
import os.path

In [4]:
#list(map(lambda x : os.path.split(x)[0], list(positive_dir.glob(r'*.jpg')))) => File Path
#list(map(lambda x : os.path.split(x)[1], list(positive_dir.glob(r'*.jpg')))) => File names 
#list(map(lambda x : os.path.split(x), list(positive_dir.glob(r'*.jpg')))) => File Path , File names 
#list(positive_dir.glob(r'*.jpg'))

In [5]:
def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

In [6]:
generate_df(positive_dir, label = 'POSITIVE')

,Filepath,Label
0,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
1,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
2,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
3,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
4,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
...,...,...
19995,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
19996,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
19997,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
19998,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE


In [7]:
generate_df(negative_dir, label = 'NEGATIVE')

,Filepath,Label
0,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
1,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
2,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
3,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
4,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
...,...,...
19995,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
19996,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
19997,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
19998,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE


In [8]:
positive_df = generate_df(positive_dir, label = 'POSITIVE')
negative_df = generate_df(negative_dir, label = 'NEGATIVE')

# SHUFFLING 
all_df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop = True)
all_df

,Filepath,Label
0,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
1,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
2,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
3,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
4,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
...,...,...
39995,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
39996,/home/pratham/study/input_data/kaggle_surface_...,NEGATIVE
39997,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE
39998,/home/pratham/study/input_data/kaggle_surface_...,POSITIVE


In [9]:
train_df, test_df = train_test_split(
    all_df.sample(6000, random_state=1),
    train_size = 0.7,
    shuffle=True,
    random_state=1
)

In [10]:
train_df.shape

(4200, 2)

In [11]:
test_df.shape

(1800, 2)

# LOAD Image Data

In [12]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    
)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

In [13]:
train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120,120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle = True,
    seed = 42,
    subset = 'training'
)
val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120,120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle = True,
    seed = 42,
    subset = 'validation'
)
test_data = test_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120,120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle = False,
    seed=42
)

Found 3360 validated image filenames belonging to 2 classes.
Found 840 validated image filenames belonging to 2 classes.
Found 1800 validated image filenames belonging to 2 classes.


# TRAINING

In [14]:
inputs = tf.keras.Input(shape=(120,120,3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 118, 118, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 59, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 57, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33    

In [15]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience = 3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
105/105 [==============================] - 6s 29ms/step - loss: 0.6847 - accuracy: 0.5614 - val_loss: 0.6483 - val_accuracy: 0.7726
Epoch 2/100
105/105 [==============================] - 3s 28ms/step - loss: 0.6275 - accuracy: 0.6991 - val_loss: 0.5651 - val_accuracy: 0.7595
Epoch 3/100
105/105 [==============================] - 3s 28ms/step - loss: 0.5234 - accuracy: 0.8057 - val_loss: 0.4914 - val_accuracy: 0.8452
Epoch 4/100
105/105 [==============================] - 3s 28ms/step - loss: 0.4208 - accuracy: 0.8829 - val_loss: 0.3893 - val_accuracy: 0.9179
Epoch 5/100
105/105 [==============================] - 3s 28ms/step - loss: 0.3328 - accuracy: 0.9212 - val_loss: 0.3109 - val_accuracy: 0.9369
Epoch 6/100
105/105 [==============================] - 3s 28ms/step - loss: 0.2670 - accuracy: 0.9366 - val_loss: 0.2625 - val_accuracy: 0.9214
Epoch 7/100
105/105 [==============================] - 3s 28ms/step - loss: 0.2009 - accuracy: 0.9494 - val_loss: 0.2241 - val_accuracy: